In [5]:
import lxml.html as lh
import codecs
#http://docs.python-guide.org/en/latest/scenarios/scrape/

riders = tree.xpath("//tr")

parse() is meant to parse from files and file-like objects, so you are
telling it to parse from the "file path" in pagecontents, which obviously
does not exist. I admit that the error message is not helpful.

In [133]:
import re
p_row = re.compile(r'<tr class="">(.*?)</tr>',re.DOTALL)
p_row2 = re.compile(r'<tr.*?>(.*?)</tr>',re.DOTALL)
p_cell = re.compile(r'<td.*?>(.*?)</td>',re.DOTALL)

p_name = re.compile(r'php">(.*?)</a>')
P_diff = re.compile(r'>(.*?)</a>')

In [63]:

mm = p_row.findall(urllib2.urlopen.read())
table = [rider(p_cell.findall(row)) for row in  mm]

print table[0].s2row()

['SAGAN Peter', 96.0, 92.8, 3.2]


In [66]:
import csv

with open('riders.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['name', 'today', 'yesterday', 'difference'])
    for x in table:
        spamwriter.writerow(x.s2row())

In [53]:
rider_scores = [ (x.diff, x.name) for x in table]

In [55]:
rider_scores.sort()

In [56]:
for x in rider_scores:
    print x

(-8.799999999999997, 'GROENEWEGEN Dylan')
(-6.9, 'CAVENDISH Mark')
(-5.799999999999999, 'GREIPEL Andr\xc3\xa9')
(-2.5, 'EWAN Caleb')
(-2.0, 'LAPORTE Christophe')
(-1.2000000000000002, 'BOASSON HAGEN Edvald')
(-0.4, 'PASQUALON Andrea')
(-0.2999999999999998, 'VAN AVERMAET Greg')
(-0.19999999999999998, 'BENOOT Tiesj')
(-0.19999999999999998, 'BERNAL Egan Arley')
(-0.19999999999999998, 'CORT  NIELSEN Magnus ')
(-0.19999999999999998, 'DE GENDT Thomas')
(-0.19999999999999998, 'DUPONT Timothy')
(-0.19999999999999998, 'EDET Nicolas')
(-0.19999999999999998, 'FRAILE Omar')
(-0.19999999999999998, 'MARCZYNSKI Tomasz')
(-0.19999999999999998, 'POLJANSKI Pawel')
(-0.19999999999999998, 'RICHEZE Ariel Maximiliano')
(-0.19999999999999998, 'SIEBERG Marcel')
(-0.19999999999999998, 'STUYVEN Jasper')
(-0.19999999999999998, 'TERPSTRA Niki')
(-0.19999999999999998, 'THOMAS Geraint')
(-0.19999999999999998, 'YATES Adam')
(-0.19999999999999996, 'FROOME Christopher')
(-0.19999999999999996, 'KWIATKOWSKI Michal')
(-0

In [62]:
class rider(object):
    
    def __init__(self,row):
        
        self.num, name_etc, pop_now, pop_then, pop_diff =   row
        
        self.name = p_name.search(name_etc).group(1)
        self.pop_now = float(pop_now)
        self.pop_then = float(pop_then)
        self.diff = self.pop_now - self.pop_then
        
    def s2row(self):
        return [self.name, self.pop_now, self.pop_then, .1*int(10*self.diff)]

            
    def __str__(self):
        return '%s %.1f'%(self.name, self.diff)


So I had to check out **requests** because of the login

http://kazuar.github.io/scraping-tutorial/

In [297]:
import requests
#hash this as we'll be using it a lot
def get_page(url):
    return session_requests.get(
            url, 
            headers = dict(referer = url))

Set up a session with the site

In [110]:
session_requests = requests.session()

base_url = 'https://www.procyclingstats.com/'
login_url = base_url + 'login.php?'

#don't keep logins in a worksheet
import passwords_web
payload = passwords_web.payload

result = session_requests.post(
    login_url, 
    data = payload, 
    headers = dict(referer=login_url)
    )

we need to get all the stages so let's look here

In [182]:
result = get_page( base_url + 'race/tour-de-france/%d/stages/winners'%2016)
mm = p_row.findall(result.text)
race_stages = [p_cell.findall(row) for row in  mm]

pick the flat non time trial stages

stages are tagged  as follows

- **t1s14** absolutely flat
- **t2s14** bumpy in the middle but flat/descent to finish

In [116]:
p_stage = re.compile(r'<a  class="" href="(.*?)">')
p_stats = re.compile('<span class="sprite stats2"></span>(.*?)</div>', re.DOTALL)
p_bold = re.compile('<b>(.*?)</b>')

In [188]:
sprint_stages = [ p_stage.search(x[1]).group(1) for x in race_stages[:] 
                                      if 't1s14' in x[0]  or 't2s14' in x[0]
                                    and 'time trial' not in x[1].lower()
                                    and '(itt)' not in x[1].lower()
]

Pull the raw pages to a pair of lists
- results from the race
- stats from the game

In [254]:
import time

result_pages = []
stat_pages = []

for xx in sprint_stages:
    url = base_url + xx
    print url
    result_pages.append(get_page(url))
    time.sleep(10)
    url +=  '/game'
    stat_pages.append(get_page(url))
    time.sleep(5)

https://www.procyclingstats.com/race/tour-de-france/2016/stage-1
https://www.procyclingstats.com/race/tour-de-france/2016/stage-3
https://www.procyclingstats.com/race/tour-de-france/2016/stage-4
https://www.procyclingstats.com/race/tour-de-france/2016/stage-6
https://www.procyclingstats.com/race/tour-de-france/2016/stage-11
https://www.procyclingstats.com/race/tour-de-france/2016/stage-14
https://www.procyclingstats.com/race/tour-de-france/2016/stage-16
https://www.procyclingstats.com/race/tour-de-france/2016/stage-21


In [191]:
stats = []
for x in stat_pages:
    snippet = p_stats.search( x.text).group(1)
    stats.append(p_bold.findall(snippet))
    
clean_stats = zip(*stats)
for k,row in enumerate(clean_stats):
    #the last line has percentages - maybe I should just calculate them
    if k == 3:
        clean_stats[k] = [float(x[:-1]) for x in row]
        continue
    clean_stats[k] = [int(x) for x in row]  

In [194]:
clean_stats

[[1048, 910, 896, 869, 806, 743, 746, 736],
 [77, 73, 55, 65, 24, 79, 57, 46],
 [559, 693, 462, 703, 629, 602, 689, 635],
 [53.3, 76.2, 51.6, 80.9, 78.0, 81.0, 92.4, 86.3]]

In [298]:
#I don't think I need the DOTALL but...
p_data = re.compile('<a href="(.*?)">.*?<a href="(.*?)">.*?'
                   '<span class="timelag">(.*?)</span>',re.DOTALL)


stage_results = []
for x in result_pages[:]:
    # a bit verbose but I was having trouble with None
    stage_results.append([])
    #because there was a gotcha in the first stage data
    #the results aren't the first table in the page
    rows = [ x for x in p_row2.findall(x.text) if 'class="time "' in x]
    for row in rows[0:20]:
        mm = p_data.search(row)  
        stage_results[-1].append( mm.groups() )

In [299]:
for i in range(8):
    print base_url + sprint_stages[i]
    for x in stage_results[i]:
        print x

https://www.procyclingstats.com/race/tour-de-france/2016/stage-1
(u'rider/mark-cavendish', u'team/dimension-data-2016', u'4:14:05')
(u'rider/marcel-kittel', u'team/etixx-quick-step-2016', u',,')
(u'rider/peter-sagan', u'team/tinkoff-2016', u',,')
(u'rider/andre-greipel', u'team/lotto-soudal-2016', u',,')
(u'rider/edward-theuns', u'team/trek-segafredo-2016', u',,')
(u'rider/christophe-laporte', u'team/cofidis-solutions-credits-2016', u',,')
(u'rider/bryan-coquard', u'team/direct-energie-2016', u',,')
(u'rider/alexander-kristoff', u'team/team-katusha-2016', u',,')
(u'rider/daniel-mclay', u'team/fortuneo-vital-concept-2016', u',,')
(u'rider/gregory-henderson', u'team/lotto-soudal-2016', u',,')
(u'rider/sondre-holst-enger', u'team/iam-cycling-2016', u',,')
(u'rider/jasper-stuyven', u'team/trek-segafredo-2016', u',,')
(u'rider/warren-barguil', u'team/team-giant-alpecin-2016', u',,')
(u'rider/julian-alaphilippe', u'team/etixx-quick-step-2016', u',,')
(u'rider/john-degenkolb', u'team/team-gia

In [290]:
top20 = []
for stage in stage_results:
    names = [ x[0].split('/')[1] for x in stage]
    top20.append(names)

In [291]:
with open('top20.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    #write the headers for the columns
    col_names = [x.split('/')[-1] for x in sprint_stages]
    spamwriter.writerow(col_names)
    #write the stats for each stage
    for row in clean_stats:
        spamwriter.writerow(row)
    #write the top 20
    for row in zip(*top20):
        spamwriter.writerow(row)